# MLB 연봉 × 성적 데이터 클리닝

**입력**: `mlb_batter_salary_stats_2021_2025.csv`, `mlb_pitcher_salary_stats_2021_2025.csv`  
**출력**: `bat_clean.csv`, `pit_clean.csv`

### 클리닝 작업 목록
1. 팀명 통일 (64종 → 30개 약어)
2. 중복 컬럼 정리 (player, season, team 중복 제거)
3. 결측률 >50% 컬럼 삭제 (희귀 구종 등)
4. years 파싱 → contract_years (숫자)
5. AAV 결측 보완 (total_value/years → salary fallback)
6. salary_M 컬럼 추가 (백만 달러 단위)

In [ ]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

DATA_DIR = './data'

bat_raw = pd.read_csv(f'{DATA_DIR}/mlb_batter_salary_stats_2021_2025.csv')
pit_raw = pd.read_csv(f'{DATA_DIR}/mlb_pitcher_salary_stats_2021_2025.csv')

print(f'타자: {bat_raw.shape}  투수: {pit_raw.shape}')
print(f'타자 시즌: {sorted(bat_raw["season"].unique())}')
print(f'투수 시즌: {sorted(pit_raw["season"].unique())}')

## 1. 팀명 통일
USA Today와 pybaseball(FanGraphs)에서 팀명 표기가 다름  
예: `"Yankees"`, `"N.Y. Yankees"`, `"NYY"` → 모두 `"NYY"`로 통일

In [ ]:
TEAM_MAP = {
    # ARI
    'Arizona': 'ARI', 'Diamondbacks': 'ARI', 'D-backs': 'ARI',
    # ATL
    'Atlanta': 'ATL', 'Braves': 'ATL',
    # BAL
    'Baltimore': 'BAL', 'Orioles': 'BAL',
    # BOS
    'Boston': 'BOS', 'Red Sox': 'BOS',
    # CHC
    'Chi. Cubs': 'CHC', 'Chic. Cubs': 'CHC', 'Chicago Cubs': 'CHC', 'Cubs': 'CHC',
    # CWS
    'Chic. White Sox': 'CWS', 'Chicago White Sox': 'CWS', 'White Sox': 'CWS',
    # CIN
    'Cincinnati': 'CIN', 'Reds': 'CIN',
    # CLE
    'Cleveland': 'CLE', 'Guardians': 'CLE', 'Indians': 'CLE',
    # COL
    'Colorado': 'COL', 'Rockies': 'COL',
    # DET
    'Detroit': 'DET', 'Tigers': 'DET',
    # HOU
    'Houston': 'HOU', 'Astros': 'HOU',
    # KC
    'Kansas City': 'KC', 'Royals': 'KC',
    # LAA
    'L.A. Angels': 'LAA', 'Angels': 'LAA', 'Los Angeles Angels': 'LAA',
    # LAD
    'L.A. Dodgers': 'LAD', 'Dodgers': 'LAD', 'Los Angeles Dodgers': 'LAD',
    # MIA
    'Miami': 'MIA', 'Marlins': 'MIA',
    # MIL
    'Milwaukee': 'MIL', 'Brewers': 'MIL',
    # MIN
    'Minnesota': 'MIN', 'Twins': 'MIN',
    # NYM
    'N.Y. Mets': 'NYM', 'Mets': 'NYM', 'New York Mets': 'NYM',
    # NYY
    'N.Y. Yankees': 'NYY', 'Yankees': 'NYY', 'New York Yankees': 'NYY',
    # OAK
    'Oakland': 'OAK', 'Athletics': 'OAK',
    # PHI
    'Philadelphia': 'PHI', 'Phillies': 'PHI',
    # PIT
    'Pittsburgh': 'PIT', 'Pirates': 'PIT',
    # SD
    'San Diego': 'SD', 'San Diego ': 'SD', 'Padres': 'SD',
    # SF
    'San Francisco': 'SF', 'Giants': 'SF',
    # SEA
    'Seattle': 'SEA', 'Mariners': 'SEA',
    # STL
    'St. Louis': 'STL', 'Cardinals': 'STL',
    # TB
    'Tampa Bay': 'TB', 'Rays': 'TB',
    # TEX
    'Texas': 'TEX', 'Rangers': 'TEX',
    # TOR
    'Toronto': 'TOR', 'Blue Jays': 'TOR',
    # WSH
    'Washington': 'WSH', 'Nationals': 'WSH',
}

def std_team(t):
    if pd.isna(t): return 'UNK'
    return TEAM_MAP.get(str(t).strip(), str(t).strip())

bat = bat_raw.copy()
pit = pit_raw.copy()

for df in [bat, pit]:
    df['team'] = df['team'].apply(std_team)
    if 'Team' in df.columns:
        df['Team'] = df['Team'].apply(std_team)

print(f'타자 팀 수: {bat_raw["team"].nunique()} → {bat["team"].nunique()}')
print(f'투수 팀 수: {pit_raw["team"].nunique()} → {pit["team"].nunique()}')
print(f'\n30개 팀: {sorted(bat["team"].unique())}')

## 2. 중복 컬럼 정리
머지 과정에서 생긴 중복 컬럼 정리  
- `player_salary`, `player_bat`, `player_pit` → `player` 하나로 통일  
- `Season`, `Year` → `season`만 유지  
- `role` → position으로 충분하므로 제거

In [ ]:
for df, label in [(bat, '타자'), (pit, '투수')]:
    before = df.shape[1]
    
    # player 통일: salary 쪽 이름 우선
    if 'player_salary' in df.columns:
        df['player'] = df['player_salary']
    
    # 불필요 컬럼 제거
    drop = [c for c in ['player_salary', 'player_bat', 'player_pit', 'Season', 'Year', 'role']
            if c in df.columns]
    df.drop(columns=drop, inplace=True)
    
    print(f'{label}: {drop} 제거 → {before} → {df.shape[1]}컬럼')

## 3. 결측률 높은 컬럼 제거 (>50%)
희귀 구종(FT, SC, KN 등), 마이너 Statcast 지표 등 대부분의 선수에게 값이 없는 컬럼 제거  
단, 분석에 중요한 컬럼(`salary`, `total_value`, `aav`, `WAR` 등)은 결측률과 무관하게 보존

In [ ]:
PROTECT = {'salary', 'total_value', 'aav', 'years', 'player', 'team',
           'position', 'season', 'name_key', 'WAR', 'Team', 'IDfg'}

for df, label in [(bat, '타자'), (pit, '투수')]:
    before = df.shape[1]
    null_rate = df.isnull().mean()
    drop_cols = [c for c in null_rate[null_rate > 0.5].index if c not in PROTECT]
    df.drop(columns=drop_cols, inplace=True)
    print(f'{label}: {len(drop_cols)}개 컬럼 제거 ({before} → {df.shape[1]})')

# 남은 결측 현황
for df, label in [(bat, '타자'), (pit, '투수')]:
    remaining = df.isnull().mean()
    still_high = remaining[(remaining > 0.1) & (remaining <= 0.5)].sort_values(ascending=False)
    if len(still_high) > 0:
        print(f'\n{label} - 10~50% 결측 컬럼 ({len(still_high)}개):')
        for col, rate in still_high.items():
            print(f'  {col}: {rate:.0%}')

## 4. years 컬럼 파싱
`"3 (2019-21)"` → `contract_years = 3` (숫자)으로 변환  
값이 없거나 `" "` (공백)인 경우 → NaN (1년 계약, pre-arb 등)

In [ ]:
def parse_contract_years(val):
    if pd.isna(val) or str(val).strip() in ['', ' ']:
        return np.nan
    m = re.match(r'(\d+)', str(val).strip())
    return int(m.group(1)) if m else np.nan

for df, label in [(bat, '타자'), (pit, '투수')]:
    df['contract_years'] = df['years'].apply(parse_contract_years)
    parsed = df['contract_years'].notna().sum()
    print(f'{label}: {parsed}/{len(df)} 파싱 성공 ({parsed/len(df):.0%})')
    df.drop(columns=['years'], inplace=True)

# 분포 확인
print('\n타자 계약 연수 분포:')
print(bat['contract_years'].value_counts().sort_index())

## 5. AAV 결측 보완
**보완 순서**:  
1. `aav`가 NaN이고 `total_value`와 `contract_years`가 있으면 → `total_value / contract_years`  
2. 그래도 NaN이면 → 해당 시즌 `salary`로 대체  

pre-arb, 중재 계약 선수는 원래 AAV가 없으므로 salary = AAV로 간주

In [ ]:
for df, label in [(bat, '타자'), (pit, '투수')]:
    before_null = df['aav'].isnull().sum()
    
    # 방법 1: total_value / contract_years
    mask = df['aav'].isnull()
    has_both = mask & df['total_value'].notna() & df['contract_years'].notna()
    df.loc[has_both, 'aav'] = (
        df.loc[has_both, 'total_value'] / df.loc[has_both, 'contract_years']
    )
    mid_null = df['aav'].isnull().sum()
    
    # 방법 2: salary fallback
    still_null = df['aav'].isnull()
    df.loc[still_null, 'aav'] = df.loc[still_null, 'salary']
    after_null = df['aav'].isnull().sum()
    
    print(f'{label} AAV 결측: {before_null} → (계산){mid_null} → (salary대체){after_null}')

## 6. 편의 컬럼 추가 + 저장

In [ ]:
# salary_M (백만 달러 단위)
for df in [bat, pit]:
    df['salary_M'] = (df['salary'] / 1e6).round(2)

# defrag (PerformanceWarning 방지)
bat = bat.copy()
pit = pit.copy()

# 저장
bat.to_csv(f'{DATA_DIR}/bat_clean.csv', index=False, encoding='utf-8-sig')
pit.to_csv(f'{DATA_DIR}/pit_clean.csv', index=False, encoding='utf-8-sig')

print('저장 완료!')
print(f'타자: {bat.shape[0]}행 * {bat.shape[1]}컬럼')
print(f'투수: {pit.shape[0]}행 * {pit.shape[1]}컬럼')

## 클리닝 결과 검증

In [ ]:
print('=== 시즌별 선수 수 ===')
print('타자:', bat['season'].value_counts().sort_index().to_dict())
print('투수:', pit['season'].value_counts().sort_index().to_dict())

print(f'\n=== 결측 현황 ===')
print(f'AAV 결측:      타자 {bat["aav"].isnull().sum()}, 투수 {pit["aav"].isnull().sum()}')
print(f'salary 결측:   타자 {bat["salary"].isnull().sum()}, 투수 {pit["salary"].isnull().sum()}')
print(f'WAR 결측:      타자 {bat["WAR"].isnull().sum()}, 투수 {pit["WAR"].isnull().sum()}')
print(f'팀 수:         {bat["team"].nunique()}')

print(f'\n=== 연봉 요약 ===')
for df, label in [(bat, '타자'), (pit, '투수')]:
    print(f'\n{label}:')
    print(f'  평균: ${df["salary_M"].mean():.1f}M')
    print(f'  중앙: ${df["salary_M"].median():.1f}M')
    print(f'  최대: ${df["salary_M"].max():.1f}M ({df.loc[df["salary_M"].idxmax(), "player"]})')

In [ ]:
# 최종 컬럼 목록 확인
print(f'타자 컬럼 ({bat.shape[1]}개):')
print(bat.columns.tolist())
print(f'\n투수 컬럼 ({pit.shape[1]}개):')
print(pit.columns.tolist())